## 1. GET PREDICTIONS FROM TRAINED MODEL

#### LOADING LIBRARIES

In [45]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import joblib
import optuna
import openpyxl

from statsmodels.tsa.seasonal import STL
from sklearn.mixture import GaussianMixture
from dtaidistance import dtw
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances
from statsmodels.nonparametric.smoothers_lowess import lowess

#### LOAD DATA

In [46]:
# Load Excel file into a DataFrame

file_path = 'Hourly_Coca_Cola_Plant.xlsx'
#file_path = 'C:/Users/Lenovo/OneDrive/Desktop/SA HAMID/SHAMS POWER/SHAMS_HOURLY AND STRING LEVEL/Hourly_Dandot _Plant2.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

#### FEATURE ENGINEERING

In [47]:
# Separate 'timestamp' into 'Date' and 'Hour'
df['Date'] = pd.to_datetime(df['timestamp']).dt.date
df['Hour'] = pd.to_datetime(df['timestamp']).dt.hour

# 4. Add 'Day of Year' column
df['Day_of_Year'] = pd.to_datetime(df['timestamp']).dt.dayofyear

# Generate cyclical features for 'Hour' and 'Day_of_Year'
df['Hour_Sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_Cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Day_Sin'] = np.sin(2 * np.pi * df['Day_of_Year'] / 365)
df['Day_Cos'] = np.cos(2 * np.pi * df['Day_of_Year'] / 365)

df['radiation_lag_1'] = df['radiation_intensity'].shift(1)
df['radiation_lag_1'] = df['radiation_lag_1'].fillna(0)

df['radiation_lag_2'] = df['radiation_intensity'].shift(2)
df['radiation_lag_2'] = df['radiation_lag_2'].fillna(0)

df['radiation_hour_interaction'] = df['radiation_intensity'] * df['Hour']
df['radiation_day_interaction'] = df['radiation_intensity'] * df['Day_of_Year']

# Calculate rate of change (difference)
df['rate_of_change'] = df['radiation_intensity'].diff().fillna(0)

# Calculate irradiance rate (percentage change)
df['irradiance_rate'] = df['radiation_intensity'].pct_change().fillna(0) * 100

# Calculate rolling mean and standard deviation (for both 2-hour and 3-hour windows)
df['rolling_mean_2h'] = df['radiation_intensity'].rolling(window=2).mean().fillna(0)
df['rolling_std_2h'] = df['radiation_intensity'].rolling(window=2).std().fillna(0)
df['rolling_mean_3h'] = df['radiation_intensity'].rolling(window=3).mean().fillna(0)
df['rolling_std_3h'] = df['radiation_intensity'].rolling(window=3).std().fillna(0)

df['Date'] = pd.to_datetime(df['Date'])                                                         # Ensure 'Date' in the main DataFrame is in datetime64[ns] format
daily_accumulated = df.groupby(df['Date'].dt.date)['radiation_intensity'].sum().reset_index()   # Group by 'Date' and calculate daily accumulated irradiance
daily_accumulated['Date'] = pd.to_datetime(daily_accumulated['Date'])                           # Convert the 'Date' column in 'daily_accumulated' to match the type in 'df'
daily_accumulated.columns = ['Date', 'daily_accumulated_irradiance']                            # Rename the column for clarity
df = pd.merge(df, daily_accumulated, how='left', on='Date')                                     # Merge back with the original DataFrame

df['ema_2h'] = df['radiation_intensity'].ewm(span=2).mean().fillna(0)
df['relative_radiation'] = df['radiation_intensity'] / df['radiation_intensity'].rolling(window=24).max().fillna(0)

In [48]:
# Replace NaN and Inf values with 0
df.replace([np.inf, -np.inf], np.nan, inplace=True)     # Replace Inf/-Inf with NaN first
df.fillna(0, inplace=True)                              # Replace all NaN values with 0

#### SELECTING REQUIRED COLUMNS

In [49]:
# Filter specified columns into a new DataFrame
#filtered_columns = ['Hour',  'radiation_intensity', 'inverter_power']

filtered_columns = ['Hour', 'Day_of_Year', 'radiation_intensity', 'inverter_power','Hour_Sin', 'Hour_Cos', 'Day_Sin','Day_Cos', 
                    'radiation_lag_1', 'radiation_lag_2', 'radiation_hour_interaction','radiation_day_interaction','relative_radiation','ema_2h',
                    'rate_of_change','irradiance_rate','rolling_mean_2h','rolling_std_2h','rolling_mean_3h','rolling_std_3h','daily_accumulated_irradiance']

filtered_df = df[filtered_columns]

#### GETTING DATA READY IN INPUT FORMAT

In [50]:
#X=filtered_df [['Hour', 'radiation_intensity']]
X=filtered_df [['Hour', 'Day_of_Year', 'radiation_intensity', 'Hour_Sin', 'Hour_Cos', 'Day_Sin','Day_Cos', 
                'radiation_lag_1', 'radiation_lag_2', 'radiation_hour_interaction','radiation_day_interaction','relative_radiation','ema_2h',
                'rate_of_change','irradiance_rate','rolling_mean_2h','rolling_std_2h','rolling_mean_3h','rolling_std_3h','daily_accumulated_irradiance']]

In [51]:
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X)

#### LOADING MODEL AND PASS DATA

In [52]:
best_model_loaded = joblib.load('Best_RF_Model_CocaCola.pkl')           # Load the saved model
y_pred_loaded = best_model_loaded.predict(X_test_scaled)                # Now you can use the loaded model for predictions

#### CREATING COLUMN OF PREDICTED VALUE

In [53]:
# Add the predicted results to the original dataframe 'df'
df['Predicted Power'] = y_pred_loaded                                   # Store predicted results in a new column

In [54]:
df.head(1)

,collectTime,stationCode,timestamp,dischargeCap,radiation_intensity,inverter_power,inverterYield,power_profit,theory_power,PVYield,...,rate_of_change,irradiance_rate,rolling_mean_2h,rolling_std_2h,rolling_mean_3h,rolling_std_3h,daily_accumulated_irradiance,ema_2h,relative_radiation,Predicted Power
0,1726858800000,NE=53278269,2024-09-21 00:00:00,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.3,0.0,0.0,4.225217


## 2. CALCULATIONS

#### HOURLY CALCULATIONS

In [55]:
# Ensure 'DateTime' column is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract the Date and Hour from the 'DateTime' column
df['Hour'] = df['timestamp'].dt.hour
df['Date'] = df['timestamp'].dt.date

# Calculate the Difference column
df['Difference'] = (df['Predicted Power'] - df['inverter_power'])

In [56]:
df.head(1)

,collectTime,stationCode,timestamp,dischargeCap,radiation_intensity,inverter_power,inverterYield,power_profit,theory_power,PVYield,...,irradiance_rate,rolling_mean_2h,rolling_std_2h,rolling_mean_3h,rolling_std_3h,daily_accumulated_irradiance,ema_2h,relative_radiation,Predicted Power,Difference
0,1726858800000,NE=53278269,2024-09-21,0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.3,0.0,0.0,4.225217,4.225217


#### DAILY CALCULATIONS

In [57]:
df['Date'] = pd.to_datetime(df['Date'])                                     # Ensure 'Date' column is in datetime format
df['Difference'] = df['Predicted Power'] - df['inverter_power']             # Calculate the 'Difference' column

In [58]:
# Group by 'Date' and calculate daily sums
df_daily_diff = df.groupby('Date')['Difference'].sum().reset_index()
df_daily_inverter_power = df.groupby('Date')['inverter_power'].sum().reset_index()
df_daily_predicted_power = df.groupby('Date')['Predicted Power'].sum().reset_index()
df_daily_radiation = df.groupby('Date')['radiation_intensity'].sum().reset_index()

# Merge all dataframes to have 'Difference', 'inverter_power', and 'Predicted Power' in the same dataframe
df_daily = df_daily_diff.merge(df_daily_inverter_power, on='Date', how='left')
df_daily = df_daily.merge(df_daily_predicted_power, on='Date', how='left')
df_daily = df_daily.merge(df_daily_radiation, on='Date', how='left')
df_daily['score'] = np.where(
    df_daily['radiation_intensity'] > 0,
    df_daily['inverter_power'] / (df_daily['radiation_intensity'] * 1000),
    np.nan
)
df_daily = df_daily.dropna(subset=['score'])            # Drop rows with NaN scores

# Add a color label for 'Difference'
df_daily['Color Label'] = np.where(
    df_daily['Difference'] > 0, 
    'Predicted > Generated', 
    'Generated > Predicted'
)

# Calculate the ratio of 'inverter_power' to 'Predicted Power' and multiply by 100
df_daily['Power Ratio (%)'] = (df_daily['inverter_power'] / df_daily['Predicted Power']) * 100
df_daily['inverter_power_no_zero'] = df_daily['inverter_power'].replace(0, np.nan)      # Replace zeros in 'inverter_power' with NaN to avoid division by zero in MAPE calculation
df_daily['MAE'] = np.abs(df_daily['Predicted Power'] - df_daily['inverter_power'])      # Calculate MAE
df_daily['MAPE'] = (np.abs(df_daily['Predicted Power'] - df_daily['inverter_power']) / df_daily['inverter_power_no_zero']) * 100    # Calculate MAPE
df_daily['MAPE'].replace([np.inf, -np.inf], np.nan, inplace=True)                       # Replace any Inf values in MAPE with NaN

# Merge RMSE back into the main df_daily dataframe
df_daily = df_daily.drop(columns=['inverter_power_no_zero'])

C:\Users\ST\AppData\Local\Temp\ipykernel_17964\2213092255.py:30: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [59]:
# An increase in the daily standard deviation of the 'score' could indicate increased variability in performance.
# Calculate rolling median and rolling standard deviation
#df_daily['rolling_mean'] = df_daily['score'].rolling(window=14).mean()
#df_daily['rolling_std'] = df_daily['score'].rolling(window=14).std()

# Calculate rate of change (first-order difference) of the score
#df_daily['score_change'] = df_daily['score'].diff()
#df_daily['change_rate'] = df_daily['score_change'].rolling(window=14).mean()

In [60]:
df_daily.head(5)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799
1,2024-09-22,9078.563733,1891.96,10970.523733,5.31,0.356301,Predicted > Generated,17.245849,9078.563733,479.849666
2,2024-09-23,29.999303,10673.52,10703.519303,5.07,2.105231,Predicted > Generated,99.719725,29.999303,0.281063
3,2024-09-24,3765.885756,5475.03,9240.915756,4.35,1.258628,Predicted > Generated,59.247700,3765.885756,68.782925
4,2024-09-25,633.692200,8117.19,8750.882200,4.13,1.965421,Predicted > Generated,92.758534,633.692200,7.806793


## 3. PLOTTING

#### ACTUAL & PREDICTED VALUES AGAINST RADIATION INTENSITY AT HOUR

In [61]:
# Prepare a DataFrame for plotting actual, predicted values, and radiation_intensity
plot_df = df[['timestamp', 'inverter_power', 'Predicted Power', 'radiation_intensity']]

# Create the figure
fig = go.Figure()

# Plot actual inverter power on primary y-axis
fig.add_trace(go.Scatter(x=plot_df['timestamp'], y=plot_df['inverter_power'], mode='lines', name='Actual Power', yaxis='y1'))

# Plot predicted inverter power on primary y-axis
fig.add_trace(go.Scatter(x=plot_df['timestamp'], y=plot_df['Predicted Power'], mode='lines', name='Predicted Power', yaxis='y1'))

# Plot radiation intensity on secondary y-axis
fig.add_trace(go.Scatter(x=plot_df['timestamp'], y=plot_df['radiation_intensity'], mode='lines', name='Radiation Intensity', yaxis='y2'))

# Update layout for secondary y-axis
fig.update_layout(
    title='ACTUAL POWER, PREDICTED POWER & RADIATION INTENSITY',   
    xaxis_title='Timestamp',
    yaxis=dict(
        title='Power (W)',
        side='left',
        rangemode='tozero',  # Ensure the primary y-axis starts from zero
        range=[0, max(plot_df['inverter_power'].max(), plot_df['Predicted Power'].max())]  # Enforcing minimum value to 0
    ),
    yaxis2=dict(
        title='Radiation Intensity (W/m²)',
        side='right',
        overlaying='y',
        rangemode='tozero',  # Ensure the secondary y-axis starts from zero
        range=[0, max(plot_df['radiation_intensity'].max(), 1)]  # Enforcing minimum value to 0
    ),
    legend=dict(x=0, y=1.1, orientation='h'),
    template='plotly_white'
)

# Show the plot
fig.show()

#### ACTUAL, PREDICTED POWER VS. ERROR AT DAY

In [62]:
# Create the bar plot for 'Difference' (Primary Axis)
fig2 = px.bar(
    df_daily, 
    x='Date', 
    y='Difference', 
    title='DAILY SUM OF DIFFERENCE BETWEEN PREDICTED & ACTUAL POWER',
    color='Color Label', 
    color_discrete_map={
        'Predicted > Generated': 'green', 
        'Generated > Predicted': 'red'
    }
)

# Add 'inverter_power' as a line (Primary Axis)
fig2.add_scatter(
    x=df_daily['Date'], 
    y=df_daily['inverter_power'], 
    mode='lines', 
    name='Total Solar Generated Power', 
    line=dict(color='purple')
)

# Add 'Predicted Power' as a line (Primary Axis)
fig2.add_scatter(
    x=df_daily['Date'],
    y=df_daily['Predicted Power'],
    mode='lines',
    name='Total Predicted Power',
    line=dict(color='grey')
)

# Add 'radiation_intensity' as a line on the Secondary Axis
fig2.add_scatter(
    x=df_daily['Date'],
    y=df_daily['radiation_intensity'],
    mode='lines',
    name='Radiation Intensity',
    line=dict(color='blue', width=2, dash='dot'),  # Blue dotted line for distinction
    yaxis='y2'                                     # Assign to secondary y-axis
)

# Add a transparent band between -500 and 500 (Primary Axis)
fig2.add_shape(
    type="rect",
    x0=df_daily['Date'].min(), x1=df_daily['Date'].max(),
    y0=-500, y1=500,
    fillcolor="rgba(0, 0, 0, 0.1)",       # Transparent fill with slight black tint
    line=dict(color="rgba(0, 0, 0, 0)")   # No border line
)

# Update layout to configure secondary y-axis and overall styling
fig2.update_layout(
    xaxis_title='Date',
    yaxis_title='Sum of Differences (W)',
    yaxis=dict(
        zeroline=True,                   # Ensure primary y-axis has a zero line
        zerolinecolor='black'            # Color of the zero line
    ),
    yaxis2=dict(
        title='Radiation Intensity',     # Title for secondary y-axis
        overlaying='y',                  # Overlay on primary y-axis
        side='right',                    # Place on the right side
        zeroline=True,                   # Ensure secondary y-axis has a zero line
        zerolinecolor='black',           # Color of the zero line
        showgrid=False                   # Hide grid lines for secondary y-axis
    ),
    template='plotly',
    plot_bgcolor='white',                # White background for the plot
    paper_bgcolor='white',               # White paper background
    legend=dict(
        x=0, y=1.1, 
        orientation='h',                 # Horizontal legend
        title_text='',                   # Remove "Color" from the legend title
        font=dict(size=12)
    ),
    height=500                           # Set the height of the chart
)

fig2.show()                              # Show the final plot

#### PERCENTAGE OF FORECAST PRODUCED

Percentage of Forecast Produced = [Actual Generated Power / Forecasted Power] * 100

In [63]:
# Create the plot
fig3 = px.bar(df_daily, x='Date', y='Power Ratio (%)', title='INVERTER POWER TO PREDICTED POWER_ %AGE OF FORECAST PRODUCED')

# Add a shaded region between 90 and 110 on the y-axis
fig3.add_shape(
    type='rect',
    x0=df_daily['Date'].min(),
    x1=df_daily['Date'].max(),
    y0=90,
    y1=110,
    fillcolor='rgba(0, 255, 0, 0.2)',  # Greenish transparent color
    line=dict(color='rgba(0, 255, 0, 0)'),
    name='Target Range (90-110)'
)

# Update layout for better styling
fig3.update_layout(
    xaxis_title='Date',
    yaxis_title='Power Ratio (%)',
    template='plotly',
    plot_bgcolor='white',                           # White background for the plot
    paper_bgcolor='white',                          # White paper background
    yaxis=dict(
        range=[0, 130],                             # Set the y-axis range between 50 and 120
    ),
    legend=dict(x=0, y=1.1, orientation='h')        # Move the legend for better readability
)
fig3.show()                                         # Show the final plot

#### ERROR METRICS

In [64]:
# Create the figure
fig = go.Figure()

# Add MAE line trace on the primary y-axis
fig.add_trace(
    go.Scatter(
        x=df_daily['Date'],
        y=df_daily['MAE'],
        mode='lines',
        name='MAE',
        line=dict(color='purple', width=2)
    )
)

# Add MAPE bar trace on the secondary y-axis
fig.add_trace(
    go.Bar(
        x=df_daily['Date'],
        y=df_daily['MAPE'],
        name='MAPE (%)',
        marker=dict(color='orange', opacity=0.5),
        yaxis='y2'
    )
)

# Update layout to set up the secondary y-axis
fig.update_layout(
    title='MAE & MAPE OVER TIME',
    xaxis=dict(
        title='Date',
        showgrid=True,                # Show vertical grid lines
        gridcolor='lightgray',        # Color of the grid lines
        gridwidth=1,                  # Width of the grid lines
    ),
    yaxis=dict(
        title='MAE',
        side='left',
        showgrid=True                 # Show horizontal grid lines for primary y-axis
    ),
    yaxis2=dict(
        title='MAPE (%)',
        overlaying='y',
        side='right',
        showgrid=False                # No horizontal grid lines for secondary y-axis
    ),
    legend=dict(
        x=0.01,
        y=1.1,
        orientation='h'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    barmode='overlay'
)

# Show the figure
fig.show()

Since multiple values are very high and are in close proximity we cannot simply take average of MAE and omit them as error from our calculation 

## 4. CALCULATE SUPPRESSION

### A) AVG. ERROR EVALUATION

#### EXCLUDE EXTREME DAYS TO CALCULATE MEAN ERROR

In [65]:
# Hyperparameter tuning using Optuna
def objective(trial, X_scaled):
    # Suggest hyperparameters
    n_components = trial.suggest_int("n_components", 2, 2)
    covariance_type = trial.suggest_categorical("covariance_type", ["full", "tied", "diag", "spherical"])

    # Fit the GMM model
    gmm_model = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=42)
    gmm_model.fit(X_scaled)

    # Compute BIC as the objective to minimize
    return gmm_model.bic(X_scaled)

In [66]:
# Normalize the data
X = df_daily['MAE'].values.reshape(-1, 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [67]:
# Run Optuna to optimize GMM hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, X_scaled), n_trials=150)

[I 2025-01-03 21:22:29,525] A new study created in memory with name: no-name-6f21c64d-5add-4ee3-84d9-7a28d66260ed
[I 2025-01-03 21:22:29,581] Trial 0 finished with value: 167.52723284264223 and parameters: {'n_components': 2, 'covariance_type': 'full'}. Best is trial 0 with value: 167.52723284264223.
[I 2025-01-03 21:22:29,600] Trial 1 finished with value: 167.52723284264232 and parameters: {'n_components': 2, 'covariance_type': 'spherical'}. Best is trial 0 with value: 167.52723284264223.
[I 2025-01-03 21:22:29,618] Trial 2 finished with value: 167.52723284264232 and parameters: {'n_components': 2, 'covariance_type': 'spherical'}. Best is trial 0 with value: 167.52723284264223.
[I 2025-01-03 21:22:29,630] Trial 3 finished with value: 201.37135973258364 and parameters: {'n_components': 2, 'covariance_type': 'tied'}. Best is trial 0 with value: 167.52723284264223.
[I 2025-01-03 21:22:29,643] Trial 4 finished with value: 201.37135973258364 and parameters: {'n_components': 2, 'covariance_

In [68]:
# Get the best model with tuned hyperparameters
best_params = study.best_params
gmm_model = GaussianMixture(n_components=best_params["n_components"], covariance_type=best_params["covariance_type"], random_state=42)
gmm_model.fit(X_scaled)

GaussianMixture(n_components=2, random_state=42)

In [69]:
# Get the log likelihood of each data point under the GMM
log_likelihood = gmm_model.score_samples(X_scaled)

In [70]:
# Create a dataframe to use with Plotly Express
df_d = pd.DataFrame({'Log Likelihood': log_likelihood})

# Plot the distribution
fig = px.histogram(
    df_d, 
    x='Log Likelihood', 
    #nbins=round(len(log_likelihood) ** 0.5),  # Sturges' rule for optimized bins
    nbins = 50,
    title='DISTRIBUTION OF LOG LIKELIHOOD SCORES',
    template='plotly_white'             # White background
)

# Customize the layout with figure size and axis labels
fig.update_layout(
    xaxis_title='Log Likelihood',
    yaxis_title='Count',
    bargap=0.05,                        # Space between bars
    width=700,                          # Set figure width
    height=350                          # Set figure height
)

# Show the figure
fig.show()

In [71]:
# Function to select optimal threshold based on log-likelihood
def optimize_threshold(log_likelihood):
    thresholds = np.linspace(np.min(log_likelihood), np.max(log_likelihood), 1000)
    best_threshold = thresholds[np.argmin(np.abs(thresholds - np.median(log_likelihood)))]  # Choose threshold around median
    return best_threshold

In [72]:
best_threshold = optimize_threshold(log_likelihood)
print("Best Threshold is:",best_threshold)

Best Threshold is: -1.060933011060794


In [73]:
# Apply the threshold to detect anomalies
relaxation_factor = 1.0
df_daily['mae anomaly'] = log_likelihood < (best_threshold*relaxation_factor)

In [74]:
# Plot the time series with anomalies highlighted using Plotly Express
fig = px.line(df_daily, x='Date', y='MAE', title="ANOMALY DETECTION in 'MAE' ", template='plotly_white')

# Change the color of the line to your desired color (e.g., blue)
fig.update_traces(line=dict(color='grey'))

# Add anomalies as a scatter plot on top of the time series, make them clearly visible
fig.add_scatter(x=df_daily.loc[df_daily['mae anomaly'], 'Date'], 
                y=df_daily.loc[df_daily['mae anomaly'], 'MAE'], 
                mode='markers', 
                marker=dict(color='red', size=8, line=dict(width=2, color='red')),
                name='Anomalies')

# Update the layout to position the legend at the bottom center
fig.update_layout(
    legend=dict(
        orientation="h",          # Horizontal orientation
        yanchor="top",            # Anchor the legend to the top of the specified position
        y=-0.2,                   # Move the legend below the plot
        xanchor="center",         # Center the legend horizontally
        x=0.5                     # Position at the center horizontally
    )
)

# Show the plot
fig.show()


In [75]:
df_daily.head(3)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE,mae anomaly
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799,False
1,2024-09-22,9078.563733,1891.96,10970.523733,5.31,0.356301,Predicted > Generated,17.245849,9078.563733,479.849666,True
2,2024-09-23,29.999303,10673.52,10703.519303,5.07,2.105231,Predicted > Generated,99.719725,29.999303,0.281063,False


#### EXCLUDE ANOMALIES AND STORE REST OF THE DATA IN NEW DATAFRAME

In [76]:
filter_dates = df_daily.loc[df_daily['mae anomaly'] == True, 'Date'].tolist()
print(filter_dates)

[Timestamp('2024-09-22 00:00:00'), Timestamp('2024-09-24 00:00:00'), Timestamp('2024-09-28 00:00:00'), Timestamp('2024-09-29 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2024-10-01 00:00:00'), Timestamp('2024-10-02 00:00:00'), Timestamp('2024-10-03 00:00:00'), Timestamp('2024-10-06 00:00:00'), Timestamp('2024-10-07 00:00:00'), Timestamp('2024-10-12 00:00:00'), Timestamp('2024-10-13 00:00:00'), Timestamp('2024-10-16 00:00:00'), Timestamp('2024-10-19 00:00:00'), Timestamp('2024-10-20 00:00:00'), Timestamp('2024-10-21 00:00:00'), Timestamp('2024-10-27 00:00:00'), Timestamp('2024-11-01 00:00:00'), Timestamp('2024-11-03 00:00:00'), Timestamp('2024-11-17 00:00:00'), Timestamp('2024-11-18 00:00:00'), Timestamp('2024-11-19 00:00:00'), Timestamp('2024-11-20 00:00:00'), Timestamp('2024-11-21 00:00:00'), Timestamp('2024-11-22 00:00:00'), Timestamp('2024-11-23 00:00:00'), Timestamp('2024-11-24 00:00:00'), Timestamp('2024-11-25 00:00:00'), Timestamp('2024-11-26 00:00:00'), Timestamp('20

In [77]:
#df_new = df_daily[~df_daily['Date'].isin(pd.to_datetime(filter_dates).date)]

filter_dates = pd.to_datetime(filter_dates)                             # Ensure filter_dates is a datetime64[ns] series
df_daily['Date'] = pd.to_datetime(df_daily['Date'])                     # Ensure df_daily['Date'] is in datetime64[ns] format
df_new = df_daily[~df_daily['Date'].isin(filter_dates)]                 # Perform the filtering

In [78]:
df_new.head(2)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE,mae anomaly
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799,False
2,2024-09-23,29.999303,10673.52,10703.519303,5.07,2.105231,Predicted > Generated,99.719725,29.999303,0.281063,False


In [79]:
df_new.shape

(39, 11)

In [80]:
# Calculate and print the median value of MAE
median_mae = df_new['MAE'].median()
print(f"Median MAE: {median_mae:.2f}")

Median MAE: 255.21


In [81]:
df_daily['Median MAE']= median_mae

In [82]:
df_daily.head(10)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE,mae anomaly,Median MAE
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799,False,255.210307
1,2024-09-22,9078.563733,1891.96,10970.523733,5.31,0.356301,Predicted > Generated,17.245849,9078.563733,479.849666,True,255.210307
2,2024-09-23,29.999303,10673.52,10703.519303,5.07,2.105231,Predicted > Generated,99.719725,29.999303,0.281063,False,255.210307
3,2024-09-24,3765.885756,5475.03,9240.915756,4.35,1.258628,Predicted > Generated,59.247700,3765.885756,68.782925,True,255.210307
4,2024-09-25,633.692200,8117.19,8750.882200,4.13,1.965421,Predicted > Generated,92.758534,633.692200,7.806793,False,255.210307
5,2024-09-26,444.940931,9077.09,9522.030931,4.57,1.986234,Predicted > Generated,95.327248,444.940931,4.901801,False,255.210307
6,2024-09-27,638.295133,6663.93,7302.225133,3.25,2.050440,Predicted > Generated,91.258895,638.295133,9.578359,False,255.210307
7,2024-09-28,-1164.381352,9916.39,8752.008648,4.22,2.349855,Generated > Predicted,113.304161,1164.381352,11.741988,True,255.210307
8,2024-09-29,8043.537806,608.34,8651.877806,4.13,0.147298,Predicted > Generated,7.031306,8043.537806,1322.210903,True,255.210307
9,2024-09-30,6457.680080,3957.43,10415.110080,5.02,0.788333,Predicted > Generated,37.997006,6457.680080,163.178631,True,255.210307


### SUPPRESSION_ELIMINATING OTHER FACTORS

Suppression = max(0, Predicted - Actual - Error Threshold)

Since prediction errors are typically ~300 units [Check MAE evaluated above], so define an acceptable error threshold (e.g., ±300 units).
This is an Error in our predicted so it should not be charged to customer.


If panels are dusty 'score' will degrade. Dusty panels are not client's responsibility so degradation in production due to dust should not be charged to client. This implies Error Threshold = MAE + Loss Due to Dust & Misc Factors

If the actual production is higher than predicted, ignore the negative suppression

So what we are charging is if all of our problems are subtracted from prediction and Prediction is still > actual then that thing is suppression i-e we are charging if [Prediction - Prediction Error - Low Production due to Dust] > Actual

In [83]:
# Assuming 'df_daily' is your DataFrame and 'median_mae' is the error threshold
df_daily['Suppression'] = (df_daily['Predicted Power'] - df_daily['inverter_power'] - median_mae).clip(lower=0)

In [84]:
df_daily.head(2)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE,mae anomaly,Median MAE,Suppression
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799,False,255.210307,0.000000
1,2024-09-22,9078.563733,1891.96,10970.523733,5.31,0.356301,Predicted > Generated,17.245849,9078.563733,479.849666,True,255.210307,8823.353426


In [85]:
# Create the figure with secondary y-axis using make_subplots
fig = go.Figure()

# Add bar chart for Total Suppression (primary y-axis)
fig.add_trace(
    go.Bar(x=df_daily['Date'], y=df_daily['Suppression'], name='Suppression', 
           yaxis='y1', marker=dict(color='rgba(245, 0, 0, 0.8)')) # rgba(246, 78, 139, 0.6)
)

# Add line chart for score (secondary y-axis)
fig.add_trace(
    go.Scatter(x=df_daily['Date'], y=df_daily['score'], name='Score', 
               mode='lines', line=dict(color='grey'), yaxis='y2')
)

# Update layout with titles, axis labels, and secondary y-axis
fig.update_layout(
    title="SCORE VS. TOTAL SUPPRESSION",
    xaxis_title="Date",
    yaxis_title="Suppression",
    yaxis2=dict(
        title="Score",
        overlaying='y',
        side='right',
        showgrid=False
    ),
    barmode='group',  # To ensure the bars and lines are not stacked
    xaxis=dict(tickformat="%Y-%m-%d"),  # Format the x-axis for Date
    template='plotly_white',  # Optional: Use a dark theme for the plot
    
    # Legend customization to place it at the bottom
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="top",    # Anchor the legend to the top
        y=-0.2,           # Position the legend below the plot
        xanchor="center", # Center the legend
        x=0.5             # Center the legend horizontally
    )
)

# Show the plot
fig.show()

#### MONTHLY SUPPRESSION

In [86]:
# Convert the 'Date' column to datetime format if not already
df_daily['Date'] = pd.to_datetime(df_daily['Date'])

# Extract the month names in order
df_daily['Month'] = df_daily['Date'].dt.month_name()  # Get full month names

# Sort by the date to maintain chronological order of months
df_daily['Month'] = pd.Categorical(
    df_daily['Month'], 
    categories=[
        'January', 'February', 'March', 'April', 'May', 
        'June', 'July', 'August', 'September', 'October', 
        'November', 'December'
    ], 
    ordered=True
)

# Group by month and calculate total suppression
df_monthly_suppression = df_daily.groupby('Month', sort=False)['Suppression'].sum().reset_index()

C:\Users\ST\AppData\Local\Temp\ipykernel_17964\2738840466.py:19: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [87]:
# Create the bar plot for total suppression month-wise
fig = px.bar(
    df_monthly_suppression, 
    x='Month', 
    y='Suppression', 
    title='Month-wise Suppression',
    labels={'Suppression': 'Total Suppression (W)', 'Month': 'Month'}, 
    template='plotly_white', 
    color='Suppression',  # Color bars based on suppression value
    color_continuous_scale='Viridis'  # Use the Viridis color scale
)

# Add values on bars
fig.update_traces(
    text=df_monthly_suppression['Suppression'].round(0), 
    textposition='outside',  # Place text outside for better visibility
    texttemplate='%{text}', 
    textfont=dict(size=12)
)

# Update layout for improved appearance
fig.update_layout(
    title=dict(
        text='Month-wise Suppression',
        x=0.5,  # Center the title
        font=dict(size=18, family='Arial, sans-serif')  # Title font style
    ),
    xaxis=dict(
        title='Month',
        showgrid=False,  # Remove gridlines for x-axis
        tickangle=45,   # Rotate x-axis labels for better readability
        tickfont=dict(size=12),  # Adjust font size
    ),
    yaxis=dict(
        title='Total Units Suppression (kW)',
        showgrid=True,  # Keep gridlines for y-axis
        gridcolor='lightgrey',  # Light grey grid color
        zeroline=True,  # Show a zero-line
        zerolinecolor='lightgrey',
        tickfont=dict(size=12)  # Adjust font size
    ),
    coloraxis_colorbar=dict(
        title="Suppression (W)",  # Add a colorbar title
        titlefont=dict(size=12),
        thickness=15,  # Adjust the thickness of the colorbar
        len=0.5  # Adjust the length of the colorbar
    ),
    plot_bgcolor='white',  # White background for the plot
    paper_bgcolor='white', # White paper background
    height=500,  # Adjust the chart height
    width=1000,  # Adjust the chart width
    margin=dict(t=50, b=100)  # Adjust margins to fit rotated labels
)

# Display the plot
fig.show()


## DYNAMIC TIME WARPING

In [88]:
# Convert 'timestamp' and 'Date' to consistent datetime formats
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['Date'] = pd.to_datetime(df['Date']).dt.date

# Provide the input date as a string variable
datelist = '2024-11-17'

# Convert input date to proper datetime.date format
input_date = pd.to_datetime(datelist).date()

# Ensure the input date is in the data
if input_date not in df['Date'].unique():
    raise ValueError(f"The provided date {datelist} is not in the dataset.")

# Group data by date
grouped = df.groupby('Date')

In [89]:
# Extract radiation intensity for each date
radiation_series = {date: group.sort_values('Hour')['radiation_intensity'].values for date, group in grouped}

# Use DTW to find similar days (hour-on-hour basis)
target_series = radiation_series[input_date]

# Compute distances (DTW scores)
distances = {
    date: dtw.distance(target_series, series)
    for date, series in radiation_series.items() if date != input_date
}

In [90]:
# Find top 3 similar days (or configurable number)
top_n = 3  # Change this value as needed
similar_dates = sorted(distances, key=distances.get)[:top_n]
print(similar_dates)

[datetime.date(2024, 10, 16), datetime.date(2024, 12, 6), datetime.date(2024, 12, 4)]


In [91]:
# Prepare DTW scores for the legend
dtw_scores = {date: distances[date] for date in similar_dates}
print(dtw_scores)

{datetime.date(2024, 10, 16): 0.17146428199482253, datetime.date(2024, 12, 6): 0.17378147196982768, datetime.date(2024, 12, 4): 0.18000000000000002}


In [92]:
# Create a subplot with three columns (horizontal layout)
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(
        'RADIATION INTENSITY BY HOUR',
        'GENERATED POWER BY HOUR',
        'PREDICTED POWER BY HOUR'
    ),
    shared_yaxes=False
)

# Add the Radiation Intensity plot (First Column)
for date in [input_date] + similar_dates:
    daily_data = grouped.get_group(date).sort_values('Hour')
    fig.add_trace(go.Scatter(x=daily_data['Hour'], y=daily_data['radiation_intensity'],
                             mode='lines', name=f"{date} (Score: {dtw_scores.get(date, 0):.2f})",
                             legendgroup=str(date)),  # Set the legend group
                  row=1, col=1)

# Add the Inverter Power plot (Second Column)
for date in [input_date] + similar_dates:
    daily_data = grouped.get_group(date).sort_values('Hour')
    fig.add_trace(go.Scatter(x=daily_data['Hour'], y=daily_data['inverter_power'],
                             mode='lines', name=f"{date} (Score: {dtw_scores.get(date, 0):.2f})",
                             legendgroup=str(date)),  # Set the legend group
                  row=1, col=2)

# Add the Predicted Power plot (Third Column)
for date in [input_date] + similar_dates:
    daily_data = grouped.get_group(date).sort_values('Hour')
    fig.add_trace(go.Scatter(x=daily_data['Hour'], y=daily_data['Predicted Power'],
                             mode='lines', name=f"{date} (Score: {dtw_scores.get(date, 0):.2f})",
                             legendgroup=str(date)),  # Set the legend group
                  row=1, col=3)

# Update layout with axis titles and legend
fig.update_layout(
    height=400, width=1200,  # Adjust width for horizontal layout
    title_text="COMPARISON OF RADIATION INTENSITY, GENERATED POWER & PREDICTED POWER",
    showlegend=True,
    legend=dict(
        orientation="h",            # Horizontal legend
        yanchor="bottom",           # Align legend to the bottom of the chart
        y=-0.3,                     # Position the legend below the charts
        xanchor="center",           # Center the legend
        x=0.5                       # Place the legend at the center horizontally
    ),
template='plotly_white'    
)

# Update x-axes and y-axes titles for each column
#fig.update_xaxes(title_text="Hour", row=1, col=1)
fig.update_yaxes(title_text="Radiation Intensity", row=1, col=1)

#fig.update_xaxes(title_text="Hour", row=1, col=2)
fig.update_yaxes(title_text="Inverter Power", row=1, col=2)

#fig.update_xaxes(title_text="Hour", row=1, col=3)
fig.update_yaxes(title_text="Predicted Power", row=1, col=3)

# Show the plot
fig.show()

In [93]:
df_daily.head(2)

,Date,Difference,inverter_power,Predicted Power,radiation_intensity,score,Color Label,Power Ratio (%),MAE,MAPE,mae anomaly,Median MAE,Suppression,Month
0,2024-09-21,-295.965992,11544.04,11248.074008,5.30,2.178121,Generated > Predicted,102.631259,295.965992,2.563799,False,255.210307,0.000000,September
1,2024-09-22,9078.563733,1891.96,10970.523733,5.31,0.356301,Predicted > Generated,17.245849,9078.563733,479.849666,True,255.210307,8823.353426,September
